# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-ec86b15169-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("Olympic explorative search") 

Consider the following exploratory information need:

> investigate the results, in terms of participations and competitions won, in the past editions of the olympics between France and Germany

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q159821`  | Olympic games | node      |
| `wd:Q10843402`| Swimmer       | node |
| `wd:Q2066131` | Athlete       | node |
| `wd:Q142`     | France        | node |
| `wd:Q183`     | Germany       | node |


Also consider


```
?a wdt:P106/wdt:P279 wd:Q2066131
```

is the BGP to retrieve all instances of **athlete**


```
?p wdt:P27 wd:Q142
```

is the BGP to retrieve all **citizens of France**

## Workload Goals

1. Identify the BGP for participating or winning some competition

2. Identify the BGP for different sports or types of competitions

3. Is there some French or German Athlete participating to any olympic event with double citizenship?

4. Compare number of participations and winning events across different dimensions
 
   4.1 In which specific sport France has more winners than Germany
   
   4.2 Who has the highest number of female winners
   
   4.3 For each country , which are the atheletes with the highest number of participations


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '929008')]


1

# Task 1

*Identify the BGP for participating or winning some competition*

In [6]:
queryString = """
SELECT COUNT(distinct ?name)
WHERE { 

?a wdt:P106 wd:Q10843402 .
?a ?x ?y .
?x <http://schema.org/name> ?name .
filter(!regex(?name,"ID"))
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '193')]


1

In [8]:
queryString = """
SELECT distinct ?name ?x
WHERE { 

?a wdt:P106 wd:Q10843402 .
?a ?x ?y .
?x <http://schema.org/name> ?name .
filter(!regex(?name,"ID"))
} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'record held'), ('x', 'http://www.wikidata.org/prop/direct/P1000')]
[('name', 'native language'), ('x', 'http://www.wikidata.org/prop/direct/P103')]
[('name', 'medical condition'), ('x', 'http://www.wikidata.org/prop/direct/P1050')]
[('name', 'occupation'), ('x', 'http://www.wikidata.org/prop/direct/P106')]
[('name', 'Munzinger Sport number'), ('x', 'http://www.wikidata.org/prop/direct/P1285')]
[('name', 'described by source'), ('x', 'http://www.wikidata.org/prop/direct/P1343')]
[('name', 'participant in'), ('x', 'http://www.wikidata.org/prop/direct/P1344')]
[('name', 'languages spoken, written or signed'), ('x', 'http://www.wikidata.org/prop/direct/P1412')]
[('name', 'affiliation'), ('x', 'http://www.wikidata.org/prop/direct/P1416')]
[('name', 'nickname'), ('x', 'http://www.wikidata.org/prop/direct/P1449')]
[('name', 'birth name'), ('x', 'http://www.wikidata.org/prop/direct/P1477')]
[('name', 'country for sport'), ('x', 'http://www.wikidata.org/prop/direct/P1532')]
[

193

In [13]:
# Checking for "partecipating in", "award recived", "victory" and "partecipant"
queryString = """
SELECT distinct ?name 
WHERE { 
?a wdt:P106 wd:Q10843402 .
?a wdt:P710 ?y .
?y <http://schema.org/name> ?name .
} limit 10
"""
# The query is the same. Only interesting thing is the "partecipant" one
print("Results")
run_query(queryString)

Results
[('name', '1927 European Aquatics Championships')]
[('name', '1926 European Aquatics Championships')]


2

In [14]:
# ANSWER to 1. Victory and partecipating in
queryString = """
SELECT COUNT(distinct ?a)
WHERE { 

?a wdt:P106/wdt:P279 wd:Q2066131 .
?a wdt:P2522|wdt:P1344 ?y .

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '164848')]


1

***

# Task 2

*Identify the BGP for different sports or types of competitions*

In [20]:
# Check properties for competitions
queryString = """
SELECT distinct ?name ?z
WHERE { 
?a wdt:P106 wd:Q10843402 .
?a wdt:P1344 ?y .
?y ?z ?k .
?z <http://schema.org/name> ?name .
filter(!regex(?name,"ID"))
} 
"""
# The query is the same. Only interesting thing is the "partecipant" one
print("Results")
run_query(queryString)

Results
[('name', 'number of participants'), ('z', 'http://www.wikidata.org/prop/direct/P1132')]
[('name', 'home venue'), ('z', 'http://www.wikidata.org/prop/direct/P115')]
[('name', 'located in the administrative territorial entity'), ('z', 'http://www.wikidata.org/prop/direct/P131')]
[('name', 'winner'), ('z', 'http://www.wikidata.org/prop/direct/P1346')]
[('name', 'motto text'), ('z', 'http://www.wikidata.org/prop/direct/P1451')]
[('name', 'logo image'), ('z', 'http://www.wikidata.org/prop/direct/P154')]
[('name', 'follows'), ('z', 'http://www.wikidata.org/prop/direct/P155')]
[('name', 'followed by'), ('z', 'http://www.wikidata.org/prop/direct/P156')]
[('name', 'headquarters location'), ('z', 'http://www.wikidata.org/prop/direct/P159')]
[('name', 'country'), ('z', 'http://www.wikidata.org/prop/direct/P17')]
[('name', 'part of the series'), ('z', 'http://www.wikidata.org/prop/direct/P179')]
[('name', 'image'), ('z', 'http://www.wikidata.org/prop/direct/P18')]
[('name', 'competition c

173

In [21]:
# Check sport
queryString = """
SELECT distinct ?name ?z
WHERE { 
?a wdt:P106 wd:Q10843402 .
?a wdt:P1344 ?y .
?y wdt:P641 ?k .
?k <http://schema.org/name> ?name .

} limit 50 
"""
# The query is the same. Only interesting thing is the "partecipant" one
print("Results")
run_query(queryString)

Results
[('name', 'multi-sport event')]
[('name', 'swimming')]
[('name', 'road bicycle racing')]
[('name', 'disabled sport')]
[('name', 'synchronized swimming')]
[('name', 'open water swimming')]
[('name', 'cliff jumping')]
[('name', 'water polo')]
[('name', 'diving')]
[('name', 'figure skating')]
[('name', "men's singles")]
[('name', 'winter sport')]
[('name', 'rowing')]
[('name', 'modern pentathlon')]
[('name', 'alpine skiing')]
[('name', 'paralympic sports')]
[('name', 'para swimming')]
[('name', 'summer sport')]
[('name', 'Olympic sport')]
[('name', 'fencing')]
[('name', 'athletics')]
[('name', 'karate')]
[('name', 'volleyball')]
[('name', 'shooting sport')]
[('name', 'association football')]
[('name', 'boxing')]
[('name', 'beach volleyball')]
[('name', 'basketball')]
[('name', 'rhythmic gymnastics')]
[('name', 'judo')]
[('name', 'archery')]
[('name', 'sailing')]
[('name', 'canoeing')]
[('name', 'cycle sport')]
[('name', 'table tennis')]
[('name', 'wrestling')]
[('name', 'gymnastic

50

Together with the query before

In [31]:
# Check subclass (not very useful) and instance of (useful). This can be considered type 
queryString = """
SELECT distinct ?name ?k
WHERE { 
?a wdt:P106 wd:Q10843402 .
?a wdt:P1344 ?y .
?y wdt:P31 ?k .
?k <http://schema.org/name> ?name .

} limit 50 
"""
print("Results")
run_query(queryString)

Results
[('name', 'events in a specific year or time period'), ('k', 'http://www.wikidata.org/entity/Q18340514')]
[('name', 'Summer Paralympic Games'), ('k', 'http://www.wikidata.org/entity/Q3327913')]
[('name', 'Pan American Games'), ('k', 'http://www.wikidata.org/entity/Q230186')]
[('name', 'Summer Olympic Games'), ('k', 'http://www.wikidata.org/entity/Q159821')]
[('name', 'multi-sport event'), ('k', 'http://www.wikidata.org/entity/Q167170')]
[('name', 'Olympic sporting event'), ('k', 'http://www.wikidata.org/entity/Q18536594')]
[('name', 'Olympic sports discipline event'), ('k', 'http://www.wikidata.org/entity/Q26132862')]
[('name', 'sports season'), ('k', 'http://www.wikidata.org/entity/Q27020041')]
[('name', 'recurrent event edition'), ('k', 'http://www.wikidata.org/entity/Q27968055')]
[('name', 'sporting event'), ('k', 'http://www.wikidata.org/entity/Q16510064')]
[('name', 'FINA World Aquatics Championships'), ('k', 'http://www.wikidata.org/entity/Q273469')]
[('name', 'project'),

50

I didn't specify that the participation had to be to a sport event

In [29]:
# school shoting? 

queryString = """
SELECT distinct ?name ?z ?aa
WHERE { 
?a wdt:P106 wd:Q10843402 .
?a wdt:P1344 ?y .
?y wdt:P31 wd:Q473853 .
?y <http://schema.org/name> ?name .
?a <http://schema.org/name> ?aa .
} limit 50 
"""
# RIP
print("Results")
run_query(queryString)

Results
[('name', 'Marjory Stoneman Douglas High School shooting'), ('aa', 'Nicholas Dworet')]


1

In [34]:
# Fast check to understand if "Olympic games" is connected to the event just with an "instance of" 
queryString = """
SELECT distinct ?name ?y
WHERE { 
?a wdt:P106 wd:Q10843402 .
?a wdt:P1344 ?y .
?y wdt:P31 wd:Q159821 .
?y <http://schema.org/name> ?name .

} limit 50 
"""
# The query is the same. Only interesting thing is the "partecipant" one
print("Results")
run_query(queryString)

Results
[('name', '1904 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8098')]
[('name', '1936 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8150')]
[('name', '1960 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8415')]
[('name', '2020 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q181278')]
[('name', '1900 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8088')]
[('name', '1972 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8438')]
[('name', '1976 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8444')]
[('name', '1988 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8470')]
[('name', '1992 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8488')]
[('name', '1996 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8531')]
[('name', '2000 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8544')]
[('name', '2004 Summer Olympics'), ('y', 'http://www.wikidata.org/entity/Q8558')]
[('nam

30

***

# Task 3

*Is there some French or German Athlete participating to any olympic event with double citizenship?*

I asked the query only about athletes having precisely the double citizenship German/French, but being a subgroup of the group requested in the task 3, the answer is the same also for the more general query about double citizenship.

In [3]:
# ANSWER to 3 
queryString = """
ASK WHERE {
SELECT distinct ?name
WHERE { 
?a wdt:P106/wdt:P279* wd:Q2066131 .
?a wdt:P1344/wdt:P31 wd:Q159821 .
?a wdt:P27 wd:Q183 .
?a wdt:P27 wd:Q142 .
?a <http://schema.org/name> ?name .

} 
}
"""
print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

***

# Task 4.1

*In which specific sport France has more winners than Germany*

In [4]:
# Winners of France
queryString = """
SELECT (count(distinct ?a) as ?num1) ?name
WHERE { 
?a wdt:P106/wdt:P279 wd:Q2066131 .
?a wdt:P27 wd:Q142 .
?a wdt:P2522|wdt:P166 ?z .
?z wdt:P641 ?sport .
?sport <http://schema.org/name> ?name.

} group by ?name
"""
print("Results")
run_query(queryString)

Results
[('num1', '1'), ('name', 'triathlon')]
[('num1', '1'), ('name', 'water polo')]
[('num1', '16'), ('name', 'fencing')]
[('num1', '15'), ('name', 'athletics')]
[('num1', '2'), ('name', 'track and field')]
[('num1', '1'), ('name', 'synchronized swimming')]
[('num1', '1'), ('name', 'cyclo-cross')]
[('num1', '6'), ('name', 'chess')]
[('num1', '1'), ('name', 'golf')]
[('num1', '1'), ('name', 'weightlifting')]
[('num1', '16'), ('name', 'volleyball')]
[('num1', '19'), ('name', 'association football')]
[('num1', '3'), ('name', 'BMX')]
[('num1', '16'), ('name', 'tennis')]
[('num1', '2'), ('name', 'badminton')]
[('num1', '5'), ('name', 'cycle sport')]
[('num1', '1'), ('name', 'ultra-trail')]
[('num1', '3'), ('name', 'judo')]
[('num1', '7'), ('name', 'petanque')]
[('num1', '3'), ('name', 'ice hockey')]
[('num1', '1'), ('name', 'windsurfing')]
[('num1', '34'), ('name', 'handball')]
[('num1', '6'), ('name', 'basketball')]
[('num1', '15'), ('name', 'road bicycle racing')]
[('num1', '1'), ('nam

31

In [3]:
# Answer
queryString = """
SELECT distinct ?name1 ?num1 ?num2 where {
{
SELECT (count(distinct ?a) as ?num1) ?name1
WHERE { 
?a wdt:P106/wdt:P279 wd:Q2066131 .
?a wdt:P27 wd:Q142 .
?a wdt:P2522|wdt:P166 ?z .
?z wdt:P641 ?sport .
?sport <http://schema.org/name> ?name1.

} group by ?name1
}{
SELECT (count(distinct ?a) as ?num2) ?name2
WHERE { 
?a wdt:P106/wdt:P279 wd:Q2066131 .
?a wdt:P27 wd:Q183 .
?a wdt:P2522|wdt:P166 ?z .
?z wdt:P641 ?sport .
?sport <http://schema.org/name> ?name2.

} group by ?name2
}
filter(?num1>?num2)
filter(?name1=?name2)
}
"""
print("Results")
run_query(queryString)

Results
[('name1', 'road bicycle racing'), ('num1', '15'), ('num2', '3')]
[('name1', 'fencing'), ('num1', '16'), ('num2', '2')]
[('name1', 'basketball'), ('num1', '6'), ('num2', '1')]
[('name1', 'chess'), ('num1', '6'), ('num2', '1')]
[('name1', 'tennis'), ('num1', '16'), ('num2', '8')]
[('name1', 'athletics'), ('num1', '15'), ('num2', '12')]
[('name1', 'cycle sport'), ('num1', '5'), ('num2', '2')]


7

***

# Task 4.2

*Who has the highest number of female winners*

In [53]:
# Winners of France
queryString = """
SELECT distinct ?name ?p ?name2 ?x {
 
?a wdt:P106/wdt:P279 wd:Q2066131 .
?a wdt:P27 wd:Q142 .
?a wdt:P2522|wdt:P166 ?z .
?a ?p ?x . 
?p <http://schema.org/name> ?name .
filter(regex(?name,"gender") || regex(?name,"sex"))
?x <http://schema.org/name> ?name2 .
}
"""
print("Results")
run_query(queryString)

Results
[('name', 'sex or gender'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('name2', 'female'), ('x', 'http://www.wikidata.org/entity/Q6581072')]
[('name', 'sex or gender'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('name2', 'male'), ('x', 'http://www.wikidata.org/entity/Q6581097')]
[('name', 'sex or gender'), ('p', 'http://www.wikidata.org/prop/direct/P21'), ('name2', 'transgender female'), ('x', 'http://www.wikidata.org/entity/Q1052281')]
[('name', 'sexual orientation'), ('p', 'http://www.wikidata.org/prop/direct/P91'), ('name2', 'homosexuality'), ('x', 'http://www.wikidata.org/entity/Q6636')]


4

In [5]:
# ANSWER to 5
queryString = """
SELECT ?france ?germany {
{
SELECT (count(distinct ?a) as ?france) 
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131 .
    ?a wdt:P27 wd:Q142 .
    ?a wdt:P2522|wdt:P166 ?z .
    ?a wdt:P21 wd:Q6581072 .
    } 
}{
SELECT (count(distinct ?a) as ?germany) 
WHERE { 
    ?a wdt:P106/wdt:P279 wd:Q2066131 .
    ?a wdt:P27 wd:Q183 .
    ?a wdt:P2522|wdt:P166 ?z .
    ?a wdt:P21 wd:Q6581072 .
    } 
}
}
"""
print("Results")
run_query(queryString)

Results
[('france', '186'), ('germany', '471')]


1

***

# Task 4.3

*For each country , which are the atheletes with the highest number of participations*

In [66]:
# Internal part of ANSWER to 6
queryString = """
SELECT (count(distinct ?x) as ?part) ?name sample(?country) {
    ?a wdt:P106/wdt:P279 wd:Q2066131 .
    ?a wdt:P27 ?country .
    ?a wdt:P1344 ?x .
    ?x wdt:P641 ?z .
    ?a <http://schema.org/name> ?name .
} group by ?name
order by desc (?part)
limit 20

"""
print("Results")
run_query(queryString)

Results
[('part', '325'), ('name', 'Monika Cassens'), ('callret-2', 'http://www.wikidata.org/entity/Q16957')]
[('part', '237'), ('name', 'Gillian Gilks'), ('callret-2', 'http://www.wikidata.org/entity/Q145')]
[('part', '211'), ('name', 'Edgar Michalowsky'), ('callret-2', 'http://www.wikidata.org/entity/Q16957')]
[('part', '208'), ('name', 'Erfried Michalowsky'), ('callret-2', 'http://www.wikidata.org/entity/Q16957')]
[('part', '183'), ('name', 'Petra Michalowsky'), ('callret-2', 'http://www.wikidata.org/entity/Q16957')]
[('part', '168'), ('name', 'Petya Nedelcheva'), ('callret-2', 'http://www.wikidata.org/entity/Q219')]
[('part', '166'), ('name', 'Usain Bolt'), ('callret-2', 'http://www.wikidata.org/entity/Q766')]
[('part', '154'), ('name', 'Angela Michalowski'), ('callret-2', 'http://www.wikidata.org/entity/Q16957')]
[('part', '147'), ('name', 'Rikke Olsen'), ('callret-2', 'http://www.wikidata.org/entity/Q35')]
[('part', '146'), ('name', 'Denyse Julien'), ('callret-2', 'http://www.wik

20

In [67]:
# ANSWER to 6
queryString = """
Select max(?part) as ?max sample(?name) ?c where{ 


SELECT (count(distinct ?x) as ?part) ?name sample(?country) as ?c {
    ?a wdt:P106/wdt:P279 wd:Q2066131 .
    ?a wdt:P27 ?cx .
    ?cx <http://schema.org/name> ?country .
    ?a wdt:P1344 ?x .
    ?x wdt:P641 ?z .
    ?a <http://schema.org/name> ?name .
} group by ?name
order by desc (?part)
}group by ?c
order by desc (?max)
limit 50

"""
print("Results")
run_query(queryString)


Results
[('max', '325'), ('callret-1', 'Monika Cassens'), ('c', 'German Democratic Republic')]
[('max', '237'), ('callret-1', 'Gillian Gilks'), ('c', 'United Kingdom')]
[('max', '168'), ('callret-1', 'Petya Nedelcheva'), ('c', 'Bulgaria')]
[('max', '166'), ('callret-1', 'Usain Bolt'), ('c', 'Jamaica')]
[('max', '147'), ('callret-1', 'Rikke Olsen'), ('c', 'Denmark')]
[('max', '146'), ('callret-1', 'Denyse Julien'), ('c', 'Canada')]
[('max', '139'), ('callret-1', 'Svetlana Zilberman'), ('c', 'Soviet Union')]
[('max', '125'), ('callret-1', 'Jürgen Koch'), ('c', 'Austria')]
[('max', '122'), ('callret-1', 'Robert Mateusiak'), ('c', 'Poland')]
[('max', '122'), ('callret-1', 'Gao Ling'), ('c', "People's Republic of China")]
[('max', '121'), ('callret-1', 'Nicole Grether'), ('c', 'Germany')]
[('max', '120'), ('callret-1', 'Lee Yong-dae'), ('c', 'South Korea')]
[('max', '109'), ('callret-1', 'Eva Twedberg'), ('c', 'Sweden')]
[('max', '106'), ('callret-1', 'Lee Chong Wei'), ('c', 'Malaysia')]
[(

50

In [69]:
# Who is for italy?
queryString = """
SELECT (count(distinct ?x) as ?part) ?name  {
    ?a wdt:P106/wdt:P279 wd:Q2066131 .
    ?a wdt:P27 ?country .
    ?country <http://schema.org/name> ?name1 .
    filter(regex(?name1,"taly"))
    ?a wdt:P1344 ?x .
    ?x wdt:P641 ?z .
    ?a <http://schema.org/name> ?name .
} group by ?name
order by desc (?part)
limit 20

"""
print("Results")
run_query(queryString)

Results
[('part', '46'), ('name', 'Klaus Raffeiner')]
[('part', '43'), ('name', 'Agnese Allegrini')]
[('part', '37'), ('name', 'Davide Rebellin')]
[('part', '36'), ('name', 'Vincenzo Nibali')]
[('part', '34'), ('name', 'Rosario Maddaloni')]
[('part', '34'), ('name', 'Gianluca Brambilla')]
[('part', '33'), ('name', 'Sonny Colbrelli')]
[('part', '33'), ('name', 'Elia Viviani')]
[('part', '33'), ('name', 'Lionel Messi')]
[('part', '32'), ('name', 'Damiano Caruso')]
[('part', '31'), ('name', 'Fabio Aru')]
[('part', '30'), ('name', 'Daniel Oss')]
[('part', '30'), ('name', 'Daniele Bennati')]
[('part', '29'), ('name', 'Domenico Pozzovivo')]
[('part', '29'), ('name', 'Giuseppe Gibilisco')]
[('part', '29'), ('name', 'Maria Luisa Mur')]
[('part', '29'), ('name', 'Alessandro De Marchi')]
[('part', '29'), ('name', 'Marco Marcato')]
[('part', '28'), ('name', 'Giovanni Visconti')]
[('part', '28'), ('name', 'Sacha Modolo')]


20